In [3]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

/Users/morganabbitt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)
/Users/morganabbitt/anaconda3/lib/python3.7/importlib/_bootstrap.py:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject
  return f(*args, **kwds)


In [15]:
def cleaning_final_df():
    #see all available columns
    pd.set_option('display.max_columns', None)
    #reads in each csv separately
    pitches = pd.read_csv('../data/pitches.csv', low_memory=False)
    atbats = pd.read_csv('../data/atbats.csv', low_memory=False)
    games = pd.read_csv('../data/games.csv', low_memory=False)
    names = pd.read_csv('../data/player_names.csv', low_memory=False)

    #merge pitches with atbats
    game_sit = pd.merge(atbats, pitches[['px', 'pz', 'start_speed', 'end_speed', 'spin_rate', 'spin_dir', 'break_angle', 'break_length', 'code', 'type', 'pitch_type', 'b_score', 'b_count', 's_count', 'outs', 'pitch_num', 'on_1b', 'on_2b', 'on_3b', 'ab_id']], on='ab_id')

    #creates full name column inside names df
    names['full_name'] = names['first_name'] + ' ' + names['last_name']
    #merges names with game situation df 
    merged_names = pd.merge(names[['full_name', 'id']], game_sit, left_on='id', right_on='batter_id')
    #merges the above merged names with the pitcher id now
    final_play_df = pd.merge(merged_names, names[['full_name', 'id']], left_on='pitcher_id', right_on='id')
    #renames some of the merged column names, and drops other id_x and id+y columns
    final_play_df = final_play_df.rename(columns={'full_name_x':'batter_name', 'g_id':'game_id', 'o' :'post_outs', 'stand':'batter_stand', 'full_name_y':'pitcher_name'}).drop(columns=['id_x', 'id_y'])
    
    #grabs certain columns to use for modeling
    final_play_df = final_play_df[['pitcher_name', 'batter_name', 'ab_id', 'batter_id', 'pitcher_id', 'game_id', 'inning', 'p_score', 'p_throws', 'batter_stand', 'top', 'b_score', 'b_count', 's_count', 'outs', 'pitch_num', 'on_1b', 'on_2b', 'on_3b', 'pitch_type']]
    final_play_df.replace({'R': 1, 'L': 0}, inplace=True)
    
    #finds all nans and uses np.where to grab axis [0] grabs first of tuple
    rows_with_nans = np.where(pitches.isna().any(axis=1) == True)[0]
    
    #uses the rows_with_nans from earlier to drop unwanted rows. only rows with nans are pitch type
    final_play_df.drop(index=rows_with_nans, axis=0, inplace=True)
    
    #uses get dummies on pitcher names to create new features
    pitch_names = pd.get_dummies(final_play_df['pitcher_name'])
    
    #merges class_modeling with the new pitch names, drops the redundant columns 
    #drops one of the pitchers from the df, Sorry 'Bronson Arroyo' you gotta go
    df = final_play_df.merge(pitch_names, left_index=True, right_index=True).drop(columns=['ab_id', 'pitcher_name', 'batter_name', 'batter_id', 'pitcher_id', 'game_id', 'Bronson Arroyo'])
    return df